In [1]:
import pandas as pd
import re

In [2]:
df=pd.read_csv("data//data_original//1113_data.csv")
df_institution=pd.read_csv("data//item_department_match//institution_match.csv")

In [3]:
# Observed from COVID disclosure report, Education. is likely Gauteng-Education
df.loc[df["Institution_name"]=="Education.","Institution_name"]="Gauteng - education"

In [4]:
df["Institution_name"]=df["Institution_name"].apply(lambda x:str(x).upper())
df["Institution_name"]=df["Institution_name"].apply(lambda x:re.sub(r"\.","",x))
df_institution["Original_name"]=df_institution["Original_name"].apply(lambda x:x.upper())

In [5]:
df_use=pd.merge(df,df_institution,how="left",left_on="Institution_name",right_on="Original_name") 
df_use.drop(columns=[" filter","Original_name"],inplace=True)

In [6]:
df_use

,No,Institution_name,Item_desp,Supplier_name,Service_desp,Unit_price,Quantity,UOM,Date_order,Order_amount,Date_payments,Payments,pay_num,order_num,Cleaned_name
0,0,WESTERN CAPE - HEALTH,Food parcels,MFJ FOODS,0,10.49,470,0,31/12/2020,"R4,930.30",2/12/2021,"R4,930",4930,4930.30,Western cape - health
1,1,WESTERN CAPE - HEALTH,Food parcels,MFJ FOODS,0,280.1,64,0,31/12/2020,"R17,926.40",2/12/2021,"R17,926",17926,17926.40,Western cape - health
2,2,KZN DEPARTMENT OF SOCIAL DEVELOPMENT,Other,Sinendaba 94 Projects Solutions,catering for Homeless people,"3,720.00",120,Each,31/12/2020,"R446,400.00",2/9/2021,"R446,400",446400,446400.00,Kwazulu natal - social development
3,3,KZN DEPARTMENT OF SOCIAL DEVELOPMENT,Other,LUTAZO TRADING ENTERPRISE,catering for Homeless people,"3,720.00",13,Each,31/12/2020,"R48,360.00",2/9/2021,"R48,360",48360,48360.00,Kwazulu natal - social development
4,4,WESTERN CAPE - HEALTH,Other,New Horizon Metals,medical Service,"1,781.58",6,0,31/12/2020,"R10,689.48",3/5/2021,"R10,689",10689,10689.48,Western cape - health
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46141,46141,BERGRIVIER MUNICIPALITY,SPAGHETTI,BOLAND SUPERSPAR,0,18,4000,Each,6/5/2020,"R151,920.00",18/06/2020,"R72,000",72000,151920.00,Bergrivier municipality
46142,46142,BERGRIVIER MUNICIPALITY,SPAGHETTI,BOLAND SUPERSPAR,0,9,4000,Each,6/5/2020,"R75,960.00",28/04/2020,"R36,000",36000,75960.00,Bergrivier municipality
46143,46143,BERGRIVIER MUNICIPALITY,VEILIGHEIDSKERMS,JACOBUS GERHARDUS DANIEL MINNAAR,0,650,3,Each,6/5/2020,"R1,950.00",18/06/2020,"R1,950",1950,1950.00,Bergrivier municipality
46144,46144,BERGRIVIER MUNICIPALITY,VEILIGHEIDSKERMS,JACOBUS GERHARDUS DANIEL MINNAAR,0,"1,300.00",3,Each,6/5/2020,"R3,900.00",29/04/2020,"R3,900",3900,3900.00,Bergrivier municipality


In [7]:
# Use cleaned Institution column for aggregation
df_agg_3level=df_use.groupby(by=["Cleaned_name","Item_desp","Service_desp"]).agg({'pay_num':'sum','order_num':'sum'})
df_agg_3level.reset_index(inplace=True)

In [8]:
# aggregate by only departments
df_agg_dep=df_use.groupby(by=["Cleaned_name"]).agg({'pay_num':'sum','order_num':'sum'})
df_agg_dep.reset_index(inplace=True)
df_agg_dep

,Cleaned_name,pay_num,order_num
0,Accounting standards board,10693,1.069353e+04
1,Airports company of south africa limited,19352308,3.547168e+07
2,All wcg departments,943931705,1.472922e+09
3,Amathole district municipality,10216916,1.558985e+07
4,Amazwi south african museum of literature,3894,3.893850e+03
...,...,...,...
478,Western cape - social development,6929322,5.354216e+07
479,Western cape - transport and public works,412153397,2.302959e+08
480,William humphreys art gallery,0,0.000000e+00
481,Witzenberg municipality,1065574,1.263573e+06


In [9]:
#Get the departments that ordered or paid most
order_top=df_agg_dep.sort_values(by="order_num",ascending=False).head(20)["Cleaned_name"].tolist()
pay_top=df_agg_dep.sort_values(by="pay_num",ascending=False).head(20)["Cleaned_name"].tolist()
institution_use=list(set(order_top+pay_top))

In [10]:
df_treemap=df_agg_3level[df_agg_3level["Cleaned_name"].isin(institution_use)]
df_treemap.reset_index(inplace=True,drop=True)
df_treemap.to_csv("data//data_viz//treemap.csv")